# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
import pickle
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///{}'.format('DisasterResponse.db'))
df = pd.read_sql_table('DisasterResponse', con=engine)

columns = df.columns[4:]
X = df['message'].values
Y = df[df.columns[4:]].values

In [3]:
X[:5]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight'], dtype=object)

In [4]:
Y[:5]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text (split text into words)
    tokens = word_tokenize(text)
    
    # Remove stop words
    words = [word for word in tokens if word not in stopwords.words('english')]

    # Lemmatization (reduce words to their root form)
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier(class_weight='balanced'))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

In [8]:
# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
pred = pipeline.predict(X_test)

precisions, recalls, f1s = [], [], []
for idx,column in enumerate(columns):
    precisions.append(precision_score(y_test[:,idx], pred[:,idx]))
    recalls.append(recall_score(y_test[:,idx], pred[:,idx]))
    f1s.append(f1_score(y_test[:,idx], pred[:,idx]))

report = pd.DataFrame({'Categories': columns, 'Precision': precisions, 'Recall': recalls, 'F1': f1s})
report

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,Categories,Precision,Recall,F1
0,related,0.766331,1.000000,0.867709
1,request,0.296566,0.258856,0.276431
2,offer,0.000000,0.000000,0.000000
3,aid_related,0.420394,1.000000,0.591940
4,medical_help,0.080073,1.000000,0.148274
5,medical_products,0.052607,1.000000,0.099956
6,search_and_rescue,0.027698,0.159341,0.047193
7,security,0.015038,0.017391,0.016129
8,military,0.000000,0.000000,0.000000
9,child_alone,0.000000,0.000000,0.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
        'tfidf__use_idf': (True, False),
        'clf__n_estimators': [10, 20]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [14]:
pred = cv.predict(X_test)

precisions, recalls, f1s = [], [], []
for idx,column in enumerate(columns):
    precisions.append(precision_score(y_test[:,idx], pred[:,idx]))
    recalls.append(recall_score(y_test[:,idx], pred[:,idx]))
    f1s.append(f1_score(y_test[:,idx], pred[:,idx]))

report = pd.DataFrame({'Categories': columns, 'Precision': precisions, 'Recall': recalls, 'F1': f1s})
report

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,Categories,Precision,Recall,F1
0,related,0.766331,1.000000,0.867709
1,request,0.216894,0.636694,0.323563
2,offer,0.000000,0.000000,0.000000
3,aid_related,0.420330,1.000000,0.591876
4,medical_help,0.080012,1.000000,0.148169
5,medical_products,0.052640,1.000000,0.100015
6,search_and_rescue,0.018519,0.005495,0.008475
7,security,0.031930,0.191304,0.054726
8,military,0.000000,0.000000,0.000000
9,child_alone,0.000000,0.000000,0.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# Build a machine learning pipeline

pipeline_abc = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1))
    ))
])

parameters_abc = {
    'tfidf__use_idf': (True, False),
    'vect__ngram_range': ((1, 1), (1, 2))
}

cv_abc = GridSearchCV(estimator=pipeline_abc, param_grid=parameters_abc, cv=3, scoring='f1_weighted')

In [16]:
# Fit the model
cv_abc.fit(X_train, y_train)

# Best parameters set
cv_abc.best_params_

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

{'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}

In [17]:
# Make predictions
pred = cv_abc.predict(X_test)

precisions, recalls, f1s = [], [], []
for idx,column in enumerate(columns):
    precisions.append(precision_score(y_test[:,idx], pred[:,idx]))
    recalls.append(recall_score(y_test[:,idx], pred[:,idx]))
    f1s.append(f1_score(y_test[:,idx], pred[:,idx]))

report = pd.DataFrame({'Categories': columns, 'Precision': precisions, 'Recall': recalls, 'F1': f1s})
report

,Categories,Precision,Recall,F1
0,related,0.770346,0.984067,0.864189
1,request,0.449664,0.121708,0.191565
2,offer,0.000000,0.000000,0.000000
3,aid_related,0.484340,0.157226,0.237390
4,medical_help,0.062500,0.001908,0.003704
5,medical_products,0.000000,0.000000,0.000000
6,search_and_rescue,0.000000,0.000000,0.000000
7,security,0.000000,0.000000,0.000000
8,military,0.000000,0.000000,0.000000
9,child_alone,0.000000,0.000000,0.000000


### 9. Export your model as a pickle file

In [18]:
with open ('classifier', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.